In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
print(os.getcwd())
os.chdir('/content/drive/My Drive/1006')
print(os.getcwd())

/content
/content/drive/My Drive/1006


In [3]:
import pandas as pd
import numpy as np
import ast
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from collections import defaultdict

In [4]:
dataset_name = 'eo'
embed_types = ['cvec_pca16', 'cvec_nmf16', 'cvec_umap16', 'cvec_tsne16', 'bert', 'roberta', 'distil', 'glove6B', 'universal']
selection_types = ['taddy_0', 'taddy_1', 'taddy_2', 'taddy_3', 'kmeans_0', 'kmeans_1', 'kmeans_2', 'kmeans_3', 'kld', 'ks', 'cos', 'recon']
counts = [100, 200, 300, 400, 700, 1000, 1400, 1800, 2400, 3000, 3600, 4200]

## Results Dict



In [13]:
key_list = ['random_noembed_0', 'random_noembed_1', 'random_noembed_2', 'random_noembed_3', 'topics_taddy_0', 'topics_taddy_1', 'topics_taddy_2', 'topics_taddy_3'] + [str(i)+'_'+str(j) for i in embed_types for j in selection_types]
acc_dict = defaultdict(list)
f1_dict = defaultdict(list)
roc_dict = defaultdict(list)

## Complete dataset

In [6]:
# Training
data = pd.read_csv(dataset_name+'_cvec_train.csv', index_col=0)
y_train = data['label'].to_numpy()
X_train = data.drop(columns=['label']).to_numpy()

# Evaluation
data = pd.read_csv(dataset_name+'_cvec_test.csv', index_col=0)
y_test = data['label'].to_numpy()
X_test = data.drop(columns=['label']).to_numpy()

del data

In [11]:
parameters = {'alpha':[1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 2, 4, 6, 8, 10, 12, 14, 16, 18]}

gscv = GridSearchCV(MultinomialNB(), parameters, verbose=1, scoring='roc_auc')
gscv.fit(X_train, y_train)
mnb = gscv.best_estimator_
print(gscv.best_params_)
print(accuracy_score(y_test, mnb.predict(X_test)))
print(f1_score(y_test, mnb.predict(X_test)))
print(roc_auc_score(y_test, mnb.predict_proba(X_test)[:,1]))

Fitting 5 folds for each of 17 candidates, totalling 85 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  85 out of  85 | elapsed:  1.1min finished


{'alpha': 16}
0.681042654028436
0.38313473877176907
0.6580137957158743


## Random Pick

In [14]:
for i in range(4):
  indices_list = []
  with open("indices_eo_random_"+str(i)+".txt") as fh: 
    lines = fh.readlines()
    for line in lines:
      indices_list.append(ast.literal_eval(line))
  for lst in indices_list:
    gscv = GridSearchCV(MultinomialNB(), parameters, verbose=0, scoring='roc_auc')
    gscv.fit(X_train[lst], y_train[lst])
    mnb = gscv.best_estimator_
    print(gscv.best_params_)
    acc_dict['random_noembed_'+str(i)].append(accuracy_score(y_test, mnb.predict(X_test)))
    f1_dict['random_noembed_'+str(i)].append(f1_score(y_test, mnb.predict(X_test)))
    roc_dict['random_noembed_'+str(i)].append(roc_auc_score(y_test, mnb.predict_proba(X_test)[:,1]))

{'alpha': 0.01}
{'alpha': 0.1}
{'alpha': 2}
{'alpha': 4}
{'alpha': 2}
{'alpha': 2}
{'alpha': 2}
{'alpha': 0.01}
{'alpha': 0.1}
{'alpha': 12}
{'alpha': 1e-07}
{'alpha': 0.1}
{'alpha': 1}
{'alpha': 2}
{'alpha': 1}
{'alpha': 2}
{'alpha': 2}
{'alpha': 4}
{'alpha': 6}
{'alpha': 0.1}
{'alpha': 4}
{'alpha': 4}
{'alpha': 8}
{'alpha': 0.001}
{'alpha': 18}
{'alpha': 1}
{'alpha': 1}
{'alpha': 0.01}
{'alpha': 2}
{'alpha': 0.1}
{'alpha': 1e-05}
{'alpha': 4}
{'alpha': 2}
{'alpha': 0.01}
{'alpha': 0.01}
{'alpha': 1e-07}
{'alpha': 12}
{'alpha': 0.01}
{'alpha': 1}
{'alpha': 2}
{'alpha': 4}
{'alpha': 2}
{'alpha': 0.1}
{'alpha': 4}
{'alpha': 2}
{'alpha': 1}
{'alpha': 1}
{'alpha': 6}


## K-means Clustering

In [15]:
for i in range(4):
  for j in range(len(embed_types)):
    indices_list = []
    with open('indices_'+dataset_name+'_'+embed_types[j]+'_kmeans_'+str(i)+'.txt') as fh:
        lines = fh.readlines()
        for line in lines:
          indices_list.append(ast.literal_eval(line))
    for lst in indices_list:
      gscv = GridSearchCV(MultinomialNB(), parameters, verbose=0, scoring='roc_auc')
      gscv.fit(X_train[lst], y_train[lst])
      mnb = gscv.best_estimator_
      acc_dict[embed_types[j]+'_kmeans_'+str(i)].append(accuracy_score(y_test, mnb.predict(X_test)))
      f1_dict[embed_types[j]+'_kmeans_'+str(i)].append(f1_score(y_test, mnb.predict(X_test)))
      roc_dict[embed_types[j]+'_kmeans_'+str(i)].append(roc_auc_score(y_test, mnb.predict_proba(X_test)[:,1]))

## Greedy farthest points based on KL Divergence

In [16]:
indices_list = []
for j in range(len(embed_types)):
  with open('indices_'+dataset_name+'_'+embed_types[j]+'_kld.txt') as fh:
    lines = fh.readlines()
    for line in lines:
      indices_list.append(ast.literal_eval(line))

In [17]:
for i, lst in enumerate(indices_list):
  for c in counts:
    gscv = GridSearchCV(MultinomialNB(), parameters, verbose=0, scoring='roc_auc')
    gscv.fit(X_train[lst[:c]], y_train[lst[:c]])
    mnb = gscv.best_estimator_
    acc_dict[embed_types[i]+'_kld'].append(accuracy_score(y_test, mnb.predict(X_test)))
    f1_dict[embed_types[i]+'_kld'].append(f1_score(y_test, mnb.predict(X_test)))
    roc_dict[embed_types[i]+'_kld'].append(roc_auc_score(y_test, mnb.predict_proba(X_test)[:,1]))

## Greedy farthest points based on Kolmogorov Smirnov statistics

In [18]:
indices_list = []
for j in range(len(embed_types)):
  with open('indices_'+dataset_name+'_'+embed_types[j]+'_ks.txt') as fh:
    lines = fh.readlines()
    for line in lines:
      indices_list.append(ast.literal_eval(line))

In [19]:
for i, lst in enumerate(indices_list):
  for c in counts:
    gscv = GridSearchCV(MultinomialNB(), parameters, verbose=0, scoring='roc_auc')
    gscv.fit(X_train[lst[:c]], y_train[lst[:c]])
    mnb = gscv.best_estimator_
    acc_dict[embed_types[i]+'_ks'].append(accuracy_score(y_test, mnb.predict(X_test)))
    f1_dict[embed_types[i]+'_ks'].append(f1_score(y_test, mnb.predict(X_test)))
    roc_dict[embed_types[i]+'_ks'].append(roc_auc_score(y_test, mnb.predict_proba(X_test)[:,1]))

## Greedy farthest points based on cosine distance

In [20]:
indices_list = []
for j in range(len(embed_types)):
  with open('indices_'+dataset_name+'_'+embed_types[j]+'_cos.txt') as fh:
    lines = fh.readlines()
    for line in lines:
      indices_list.append(ast.literal_eval(line))

In [21]:
for i, lst in enumerate(indices_list):
  for c in counts:
    gscv = GridSearchCV(MultinomialNB(), parameters, verbose=0, scoring='roc_auc')
    gscv.fit(X_train[lst[:c]], y_train[lst[:c]])
    mnb = gscv.best_estimator_
    acc_dict[embed_types[i]+'_cos'].append(accuracy_score(y_test, mnb.predict(X_test)))
    f1_dict[embed_types[i]+'_cos'].append(f1_score(y_test, mnb.predict(X_test)))
    roc_dict[embed_types[i]+'_cos'].append(roc_auc_score(y_test, mnb.predict_proba(X_test)[:,1]))

## Reconstruction Loss Minimization

In [22]:
indices_list = []
for j in range(len(embed_types)):
  indices_list.append(list(np.load('indices_'+dataset_name+'_'+embed_types[j]+'_recon.npy')))

In [23]:
for i, lst in enumerate(indices_list):
  for c in counts:
    gscv = GridSearchCV(MultinomialNB(), parameters, verbose=0, scoring='roc_auc')
    gscv.fit(X_train[lst[:c]], y_train[lst[:c]])
    mnb = gscv.best_estimator_
    acc_dict[embed_types[i]+'_recon'].append(accuracy_score(y_test, mnb.predict(X_test)))
    f1_dict[embed_types[i]+'_recon'].append(f1_score(y_test, mnb.predict(X_test)))
    roc_dict[embed_types[i]+'_recon'].append(roc_auc_score(y_test, mnb.predict_proba(X_test)[:,1]))

## Taddy

In [57]:
for k in range(4):
  for i in range(len(embed_types)):
    indices_list = []
    with open('indices_'+dataset_name+'_'+embed_types[i]+'_taddy_'+str(k)+'.txt') as fh:
      lines = fh.readlines()
      for line in lines:
        indices_list.append(ast.literal_eval(line))
    for j in range(len(indices_list[0])):
      gscv = GridSearchCV(MultinomialNB(), parameters, verbose=0, scoring='roc_auc')
      gscv.fit(X_train[indices_list[0][j]], y_train[indices_list[0][j]])
      mnb = gscv.best_estimator_
      acc_dict[embed_types[i]+'_taddy_'+str(k)].append(accuracy_score(y_test, mnb.predict(X_test)))
      f1_dict[embed_types[i]+'_taddy_'+str(k)].append(f1_score(y_test, mnb.predict(X_test)))
      roc_dict[embed_types[i]+'_taddy_'+str(k)].append(roc_auc_score(y_test, mnb.predict_proba(X_test)[:,1]))

## Taddy Topics

In [68]:
for k in range(4):
  indices_list = []
  with open('indices_'+dataset_name+'_topics_taddy_'+str(k)+'.txt') as fh:
    lines = fh.readlines()
    for line in lines:
      indices_list.append(ast.literal_eval(line))
  for j in range(len(indices_list[0])):
    gscv = GridSearchCV(MultinomialNB(), parameters, verbose=0, scoring='roc_auc')
    gscv.fit(X_train[indices_list[0][j]], y_train[indices_list[0][j]])
    mnb = gscv.best_estimator_
    acc_dict['topics_taddy_'+str(k)].append(accuracy_score(y_test, mnb.predict(X_test)))
    f1_dict['topics_taddy_'+str(k)].append(f1_score(y_test, mnb.predict(X_test)))
    roc_dict['topics_taddy_'+str(k)].append(roc_auc_score(y_test, mnb.predict_proba(X_test)[:,1]))

In [70]:
pd.DataFrame.from_dict(roc_dict, orient='index').transpose()

,random_noembed_0,random_noembed_1,random_noembed_2,random_noembed_3,cvec_pca16_kmeans_0,cvec_nmf16_kmeans_0,cvec_umap16_kmeans_0,cvec_tsne16_kmeans_0,bert_kmeans_0,roberta_kmeans_0,distil_kmeans_0,glove6B_kmeans_0,universal_kmeans_0,cvec_pca16_kmeans_1,cvec_nmf16_kmeans_1,cvec_umap16_kmeans_1,cvec_tsne16_kmeans_1,bert_kmeans_1,roberta_kmeans_1,distil_kmeans_1,glove6B_kmeans_1,universal_kmeans_1,cvec_pca16_kmeans_2,cvec_nmf16_kmeans_2,cvec_umap16_kmeans_2,cvec_tsne16_kmeans_2,bert_kmeans_2,roberta_kmeans_2,distil_kmeans_2,glove6B_kmeans_2,universal_kmeans_2,cvec_pca16_kmeans_3,cvec_nmf16_kmeans_3,cvec_umap16_kmeans_3,cvec_tsne16_kmeans_3,bert_kmeans_3,roberta_kmeans_3,distil_kmeans_3,glove6B_kmeans_3,universal_kmeans_3,...,cvec_pca16_taddy_0,cvec_nmf16_taddy_0,cvec_umap16_taddy_0,cvec_tsne16_taddy_0,bert_taddy_0,roberta_taddy_0,distil_taddy_0,glove6B_taddy_0,universal_taddy_0,cvec_pca16_taddy_1,cvec_nmf16_taddy_1,cvec_umap16_taddy_1,cvec_tsne16_taddy_1,bert_taddy_1,roberta_taddy_1,distil_taddy_1,glove6B_taddy_1,universal_taddy_1,cvec_pca16_taddy_2,cvec_nmf16_taddy_2,cvec_umap16_taddy_2,cvec_tsne16_taddy_2,bert_taddy_2,roberta_taddy_2,distil_taddy_2,glove6B_taddy_2,universal_taddy_2,cvec_pca16_taddy_3,cvec_nmf16_taddy_3,cvec_umap16_taddy_3,cvec_tsne16_taddy_3,bert_taddy_3,roberta_taddy_3,distil_taddy_3,glove6B_taddy_3,universal_taddy_3,topics_taddy_0,topics_taddy_1,topics_taddy_2,topics_taddy_3
0,0.565183,0.536345,0.532701,0.518372,0.524512,0.511829,0.540173,0.541613,0.559350,0.577143,0.591148,0.585124,0.518755,0.558481,0.597719,0.567513,0.502694,0.527444,0.600282,0.560546,0.593036,0.586988,0.497565,0.539280,0.582689,0.489051,0.562689,0.557821,0.558895,0.600469,0.527074,0.584392,0.606555,0.551708,0.588458,0.481823,0.535507,0.545239,0.517032,0.576083,...,0.586407,0.562914,0.438996,0.550648,0.559541,0.576119,0.519460,0.549146,0.590099,0.590388,0.563829,0.474972,0.500224,0.480603,0.603232,0.521142,0.547662,0.519331,0.562811,0.565269,0.481084,0.519816,0.573588,0.497968,0.492268,0.536399,0.589829,0.590862,0.558471,0.477121,0.593102,0.593532,0.505199,0.599818,0.561339,0.573470,0.552210,0.558492,0.553260,0.516108
1,0.563593,0.614749,0.521320,0.623877,0.580283,0.568702,0.597014,0.585970,0.617988,0.529856,0.507213,0.550492,0.541706,0.615791,0.571312,0.575906,0.594621,0.581187,0.575307,0.573154,0.593418,0.602914,0.577990,0.545504,0.577457,0.531563,0.566366,0.531806,0.626497,0.611455,0.603992,0.593978,0.547227,0.598164,0.620625,0.550789,0.633744,0.540944,0.604064,0.526541,...,0.579755,0.568065,0.532433,0.588658,0.556468,0.588265,0.544278,0.550132,0.570911,0.578421,0.566468,0.521153,0.594449,0.521119,0.608249,0.578422,0.501916,0.585404,0.574959,0.565127,0.576587,0.590974,0.554476,0.525830,0.541670,0.587258,0.586327,0.578176,0.566479,0.514649,0.594321,0.628544,0.562001,0.570077,0.562104,0.592035,0.507689,0.515044,0.526928,0.526428
2,0.596559,0.548057,0.622774,0.598999,0.590877,0.616259,0.596248,0.630045,0.595872,0.614715,0.587593,0.607910,0.589584,0.606526,0.585440,0.595353,0.605629,0.611956,0.559677,0.583918,0.620643,0.589870,0.612836,0.573875,0.610291,0.617412,0.561441,0.553533,0.601569,0.608552,0.611110,0.606750,0.607471,0.536760,0.629760,0.569241,0.529391,0.617054,0.565979,0.591945,...,0.554302,0.575230,0.530377,0.576128,0.617651,0.537598,0.574878,0.530935,0.578527,0.556529,0.576649,0.530954,0.590396,0.622920,0.621291,0.594013,0.566611,0.610350,0.555518,0.573614,0.525850,0.597827,0.547410,0.596566,0.548865,0.607796,0.526421,0.555642,0.575145,0.575045,0.565824,0.583242,0.538228,0.587762,0.596384,0.570661,0.513305,0.506155,0.549689,0.513699
3,0.609832,0.583116,0.584104,0.596880,0.591788,0.599731,0.536426,0.614315,0.613372,0.626276,0.570886,0.603536,0.618418,0.633536,0.605144,0.568868,0.624693,0.583113,0.569482,0.617639,0.624047,0.626834,0.631401,0.580647,0.623563,0.581508,0.641419,0.621614,0.558980,0.604829,0.583084,0.606902,0.610603,0.610939,0.582395,0.572634,0.626566,0.621549,0.613249,0.628475,...,0.564641,0.570084,0.539841,0.606200,0.6

## Save dicts to csv

In [71]:
pd.DataFrame.from_dict(acc_dict, orient='index').transpose().to_csv('eo_acc_mnb.csv')
pd.DataFrame.from_dict(f1_dict, orient='index').transpose().to_csv('eo_f1_mnb.csv')
pd.DataFrame.from_dict(roc_dict, orient='index').transpose().to_csv('eo_roc_mnb.csv')